# 🚗🚙🚘🛻 Geração de Clientes pelo eVED
Este notebook gera os clientes para os testes de aprendizado federado a partir dos dados do [eVED](https://huggingface.co/datasets/jwsouza13/eVED-train-test)

In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset

⚠️ Se tiver rodado o arquivo `eVED-train-test-generator.py`, carregue (*substituindo 40 e 42 pelos valores testados*):

In [2]:
eved_train = load_dataset("eVED_train_test_FRAC-TEST_40_STATE_42", split="train")
eved_test = load_dataset("eVED_train_test_FRAC-TEST_40_STATE_42", split="test")

In [3]:
df_eved = eved_train.to_pandas()
df_eved.head()

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Intersection,Bus Stops,Focus Points,Vehicle Type,Vehicle Class,Engine Configuration & Displacement,Transmission,Drive Wheels,Generalized_Weight,EngineType
0,1.586651,8.0,706.0,0.0,42.277558,-83.69875,40.0,22.129999,2285.0,49.019608,...,NaN,NaN,None,ICE,Car,4-FI 1.5L,5-SP MANUAL,NO DATA,2500.0,None
1,1.586651,8.0,706.0,200.0,42.277558,-83.69875,40.0,22.129999,2285.0,67.450981,...,NaN,NaN,None,ICE,Car,4-FI 1.5L,5-SP MANUAL,NO DATA,2500.0,None
2,1.586651,8.0,706.0,1100.0,42.277558,-83.69875,45.0,22.129999,2285.0,67.450981,...,NaN,NaN,None,ICE,Car,4-FI 1.5L,5-SP MANUAL,NO DATA,2500.0,None
3,1.586651,8.0,706.0,2100.0,42.277558,-83.69875,47.0,6.150000,2744.0,67.450981,...,NaN,NaN,None,ICE,Car,4-FI 1.5L,5-SP MANUAL,NO DATA,2500.0,None
4,1.586651,8.0,706.0,4200.0,42.277558,-83.69875,48.0,21.440001,1982.0,67.450981,...,NaN,NaN,None,ICE,Car,4-FI 1.5L,5-SP MANUAL,NO DATA,2500.0,None


In [14]:
df_eved.DayNum.value_counts().tail(20)

DayNum
170.982051    101
212.581255    101
160.583202    101
353.109107    101
236.708951    101
218.762512    101
19.642195     101
240.682280    101
148.802303    101
127.745507    101
35.790547     101
342.825872    101
195.950324    101
3.807485      101
163.696342    101
135.814046    101
201.727038    101
33.663761     101
242.850828    101
374.661491    101
Name: count, dtype: int64

🤗 Se quiser testar com os datasets já carregados do HF🤗:

In [ ]:
#eved_train = load_dataset("jwsouza13/eVED-train-test", split="train")
#eved_test = load_dataset("jwsouza13/eVED-train-test", split="test")

# 🧑‍🦲👩‍🦳👳‍♀️👨‍🦲👩‍🦱👶 CLIENTES  🚓🚗🚕🛺🚙🚛

In [ ]:
df_train = eved_train.to_pandas()
clients_train = eved_train.unique("VehId")
i = 0
print("Processando TREINO...")
for id in clients_train:
    folder_path = f"EVED_Clients/train/client_{i}"
    os.makedirs(folder_path, exist_ok=True) # exist_ok=True é mais seguro
    
    df_client = df_train[df_train.VehId == id].copy()
    
    # Salvar Metadados (Treino)
    veh_type = 'Unknown'

    # EngineType (EV, PHEV)
    if 'EngineType' in df_client.columns and pd.notna(df_client['EngineType'].iloc[0]):
        type_from_col = df_client['EngineType'].iloc[0]
        if type_from_col in ('EV', 'PHEV'):
            veh_type = type_from_col

    # Se não for, checar Vehicle Type (ICE, HEV)
    elif 'Vehicle Type' in df_client.columns and pd.notna(df_client['Vehicle Type'].iloc[0]):
        type_from_col = df_client['Vehicle Type'].iloc[0]
        if type_from_col in ('ICE', 'HEV'):
            veh_type = type_from_col

    metadata = {
        'VehId': id, # O ID real do veículo
        'ClientIndex': i, # O índice da pasta (client_i)
        'VehicleType': veh_type # O tipo determinado
    }
    
    with open(os.path.join(folder_path, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=4)

    for trip in df_client.Trip.unique():
        df_trip = df_client[df_client.Trip == trip].copy()
        df_trip.to_parquet(os.path.join(folder_path, f"trip_{trip}.parquet"))
        
    del df_client
    i += 1

print("Processando TESTE...")
df_test = eved_test.to_pandas()
clients_test = eved_test.unique("VehId")
i = 0
for id in clients_test:
    folder_path = f"EVED_Clients/test/client_{i}"
    os.makedirs(folder_path, exist_ok=True)
    
    df_client = df_test[df_test.VehId == id].copy()
    
    # Salvar Metadados (Teste)
    veh_type = 'Unknown'
    if 'EngineType' in df_client.columns and pd.notna(df_client['EngineType'].iloc[0]):
        type_from_col = df_client['EngineType'].iloc[0]
        if type_from_col in ('EV', 'PHEV'):
            veh_type = type_from_col
    elif 'Vehicle Type' in df_client.columns and pd.notna(df_client['Vehicle Type'].iloc[0]):
        type_from_col = df_client['Vehicle Type'].iloc[0]
        if type_from_col in ('ICE', 'HEV'):
            veh_type = type_from_col
            
    metadata = {
        'VehId': id,
        'ClientIndex': i,
        'VehicleType': veh_type
    }
    
    with open(os.path.join(folder_path, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=4)
    
    for trip in df_client.Trip.unique():
        df_trip = df_client[df_client.Trip == trip].copy()
        df_trip.to_parquet(os.path.join(folder_path, f"trip_{trip}.parquet"))
        
    del df_client
    i += 1

print("Finalizado com sucesso!")

Carregando datasets...
Processando Treino...
Processando Teste...
Processamento de dados concluído com metadados!
